<a href="https://colab.research.google.com/github/harnalashok/deeplearning/blob/main/pretrained_layers_autoencoder_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/harnalashok/deeplearning/blob/main/pretrained_layers_autoencoder_I.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

In [ ]:
# Last amended: 22nd Jan, 2021
# Myfolder: https://github.com/harnalashok/keras
# Objectives
#            i) Building autoencoder using Model class subclassing
#           ii) Using pre-trained autoencoder layers in a classifier
#          iii) Comparing Classifer performance with and without pre-trained 
#           iv) Using keras model as a layer
#            v) A pre-trained model using autoencoder gives better classification
#
# Ref: https://www.tensorflow.org/tutorials/generative/autoencoder#first_example_basic_autoencoder
#      https://www.tensorflow.org/tutorials/generative/autoencoder#third_example_anomaly_detection
#      Practical Recommendations for Gradient-Based Training of DeepArchitectures by Yoshua Bengio

In [ ]:
# 1.0 Import libraries
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

In [ ]:
# 1.1 Display outputs from multiple commands in a colab cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
# 2.0 Get fashion mnist data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize data
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Data shape
print (x_train.shape)
print (x_test.shape)


4423680/4422102 [==============================] - 0s 0us/step
(60000, 28, 28)
(10000, 28, 28)


In [ ]:
# 2.1 Reshape data for feeding it to NN model
x_train = x_train.reshape((-1, 784))
x_test = x_test.reshape((-1, 784))

In [ ]:
# 2.2 Design an Autoencoder with Subclassing
#     Ref: https://www.tensorflow.org/guide/keras/custom_layers_and_models
#     Page 313, Book: Hands-on Machine Learning witgh Scitkit-Learn, Keras, and Tensorflow

latent_dim = 64 
class Autoencoder(Model):

  # 2.2.1 Design all layers
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = 100
    # 2.2.2 This is our encoder
    self.encoder = tf.keras.Sequential(
                                        [
                                          layers.Input(shape=(784,)),
                                          layers.Dense(self.latent_dim, activation='relu'),
                                          layers.Dense(self.latent_dim, activation='relu'),
                                          #layers.GaussianNoise(0.1),
                                          layers.Dense(self.latent_dim, activation='relu')
                                        ]
                                       )
    # 2.2.3 This is our decoder
    self.decoder = tf.keras.Sequential(
                                        [
                                          layers.Dense(self.latent_dim, activation='relu'),
                                          layers.Dense(self.latent_dim, activation='relu'),
                                          layers.Dense(784, activation='sigmoid'),
                                          #layers.Reshape((28, 28))
                                         ]
                                       )
  
  # 2.2.4 Call function with just one parameter    
  def call(self, inputs):
    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded




In [ ]:
# 3.0 Instantiate, compile and train autoencoder
autoencoder = Autoencoder(100)
autoencoder.compile(optimizer='adam', loss="mse")
autoencoder.fit(x_train, x_train,
                epochs=100,
                shuffle=True,
                validation_data=(x_test, x_test))


Epoch 1/100
1875/1875 [==============================] - 9s 4ms/step - loss: 0.0385 - val_loss: 0.0178
Epoch 2/100
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0168 - val_loss: 0.0151
Epoch 3/100
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0143 - val_loss: 0.0132
Epoch 4/100
1875/1875 [==============================] - 8s 5ms/step - loss: 0.0130 - val_loss: 0.0124
Epoch 5/100
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0123 - val_loss: 0.0119
Epoch 6/100
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0117 - val_loss: 0.0113
Epoch 7/100
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0112 - val_loss: 0.0110
Epoch 8/100
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0108 - val_loss: 0.0107
Epoch 9/100
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0105 - val_loss: 0.0104
Epoch 10/100
1875/1875 [==============================] - 8s 4ms/step - l

In [ ]:
# 3.1 Layer-wise summary
print("\nSummary")
print("============")
autoencoder.summary()
# 3.1.1
print("\n\nLayers in autoencoder")
print("============")
autoencoder.layers


Summary
Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 100)               98700     
_________________________________________________________________
sequential_1 (Sequential)    (None, 784)               99384     
Total params: 198,084
Trainable params: 198,084
Non-trainable params: 0
_________________________________________________________________


Layers in autoencoder


In [ ]:
# 3.2 Just look at layers
autoencoder.layers 


# Perform classification using pre-trained weights

In [ ]:
# 4.0 Define Classification model function
#     
def class_model(trainable = False):
  model1 = tf.keras.models.Sequential()
  # 4.1 Add autoencoder as a layer
  #     But only the 'enmcoder' part
  model1.add(autoencoder.layers[-2])
  # 4.2 This is the output layer of our model
  model1.add(layers.Dense(10,activation = "softmax"))
  # 4.3 No training for autoencoder
  autoencoder.layers[-2].trainable = trainable
  model1.layers[0].trainable = trainable
  return model1

In [ ]:
# 5.0 Instantiate classification model and train it
model1 = class_model(False)
# 5.1
model1.compile(loss = "sparse_categorical_crossentropy", metrics = "accuracy")
# 5.2
model1.fit(x_train, y_train,
                epochs=100,
                shuffle=True,
                validation_data=(x_test, y_test)
                )

Epoch 1/100
1875/1875 [==============================] - 3s 2ms/step - loss: 1.2779 - accuracy: 0.5760 - val_loss: 0.5459 - val_accuracy: 0.8049
Epoch 2/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.5065 - accuracy: 0.8183 - val_loss: 0.5032 - val_accuracy: 0.8198
Epoch 3/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4702 - accuracy: 0.8314 - val_loss: 0.4947 - val_accuracy: 0.8263
Epoch 4/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4554 - accuracy: 0.8378 - val_loss: 0.4802 - val_accuracy: 0.8314
Epoch 5/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4473 - accuracy: 0.8397 - val_loss: 0.4732 - val_accuracy: 0.8344
Epoch 6/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4339 - accuracy: 0.8456 - val_loss: 0.4774 - val_accuracy: 0.8293
Epoch 7/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4412 - accuracy: 0.8438 - val_loss: 0.4755 - val_ac

In [ ]:
# 5.3 Evaluate model
model1.evaluate(x_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.4830 - accuracy: 0.8383


[0.482998251914978, 0.8382999897003174]

In [ ]:
# 5.4 Also get its summary
model1.summary()

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_36 (Sequential)   (None, 100)               98700     
_________________________________________________________________
dense_74 (Dense)             (None, 10)                1010      
Total params: 99,710
Trainable params: 1,010
Non-trainable params: 98,700
_________________________________________________________________


In [ ]:
# 6.0 Run the classification model again but 
#     this time train the autoencoder layer

model2 = class_model(True)
# 6.1
model2.compile(loss = "sparse_categorical_crossentropy", metrics = "accuracy")
# 6.2
model2.fit(x_train, y_train,
                epochs=100,
                shuffle=True,
                validation_data=(x_test, y_test)
                )

Epoch 1/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.6101 - accuracy: 0.7876 - val_loss: 0.4078 - val_accuracy: 0.8542
Epoch 2/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3718 - accuracy: 0.8666 - val_loss: 0.3709 - val_accuracy: 0.8703
Epoch 3/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3360 - accuracy: 0.8811 - val_loss: 0.3795 - val_accuracy: 0.8715
Epoch 4/100
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3381 - accuracy: 0.8791 - val_loss: 0.4454 - val_accuracy: 0.8510
Epoch 5/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3377 - accuracy: 0.8826 - val_loss: 0.4432 - val_accuracy: 0.8696
Epoch 6/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3347 - accuracy: 0.8839 - val_loss: 0.4043 - val_accuracy: 0.8669
Epoch 7/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3344 - accuracy: 0.8843 - val_loss: 0.6471 - val_ac

In [ ]:
# 6.3 Evaluate the model
#    Observe that a pre-trained model using
#    autoencoder gives better classification
model2.evaluate(x_test,y_test)

313/313 [==============================] - 0s 1ms/step - loss: 2.5922 - accuracy: 0.8587


[2.5922441482543945, 0.8586999773979187]

In [ ]:
# 7.0 If you evaluate model1 again, we get very low accuracy
#     as autoencoder weights have changed
model1.evaluate(x_test,y_test)

313/313 [==============================] - 0s 1ms/step - loss: 796.5132 - accuracy: 0.1296


[796.5132446289062, 0.12960000336170197]

In [ ]:
###### I am done ##############